In [13]:
!pip install --upgrade selenium

In [19]:
!apt-get update
!apt-get install -y chromium-browser
!apt install chromium-chromedriver

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [40]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [21]:
!ls /usr/lib/chromium-browser/chromedriver

chromedriver


In [151]:
from google.colab import userdata
email= userdata.get('email')
password= userdata.get('password')

In [152]:
url= 'https://archiveofourown.org/users/Sw33tBl0550m/readings'

def web_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--verbose")
    options.add_argument('--no-sandbox')
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument("--window-size=1920, 1200")
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=options)
    return driver

In [153]:
driver = web_driver()
driver.get(url)


In [154]:
driver.find_element(By.XPATH, '//*[@id="user_login"]').send_keys(email)
driver.find_element(By.XPATH, '//*[@id="user_password"]').send_keys(password)
driver.find_element(By.XPATH, '//*[@id="new_user"]/dl/dd[4]/input').click()

In [155]:
import pandas as pd
import time
def scrape_titles_two_pages(driver):
    titles = []
    links= []
    authors= []
    fandom_tags= []
    relationship_tags= []
    character_tags= []
    freeform_tags= []
    summary= []
    words= []
    chapters= []
    latest_view= []
    current_page = 1
    max_pages = 4

    while current_page <= max_pages:
        print(f"Scraping page {current_page}")

        # Scrape current page
        # getting titles and links
        header_titles = driver.find_elements(By.CSS_SELECTOR, 'h4.heading a[href^="/works/"]')
        print(f"Found {len(header_titles)} titles on this page")

        for title in header_titles:
            titles.append(title.text)
            links.append(title.get_attribute('href'))


        # getting authors, fandoms tags, relationship tags, character tags
        articles = driver.find_elements(By.CSS_SELECTOR, 'li[role="article"]')
        for article in articles:
            #authors
            author_elements= article.find_elements(By.CSS_SELECTOR, 'h4.heading a[rel="author"]')
            author_list= [author.text for author in author_elements]
            if not author_list:
              authors.append("Anonymous")
            else:
              authors.append(", ".join(author_list))

            # fandoms
            fandom_elements = article.find_elements(By.CSS_SELECTOR, 'h5.fandoms a')
            fandom_list= [fandom.text for fandom in fandom_elements]
            fandom_tags.append(", ".join(fandom_list))

            # relationship tags
            relationship_elements= article.find_elements(By.CSS_SELECTOR, 'li.relationships a')
            relationship_list= [relationship.text for relationship in relationship_elements]
            relationship_tags.append(", ".join(relationship_list))

            # character tags
            character_elements= article.find_elements(By.CSS_SELECTOR, 'li.characters a')
            character_list= [character.text for character in character_elements]
            character_tags.append(", ".join(character_list))

            # freeforms
            freeform_elements= article.find_elements(By.CSS_SELECTOR, 'li.freeforms a')
            freeform_list= [freeform.text for freeform in freeform_elements]
            freeform_tags.append(", ".join(freeform_list))

            # summary
            summary_elements= article.find_elements(By.CSS_SELECTOR, 'blockquote.userstuff p')
            summary_list= [summary.text for summary in summary_elements]
            summary.append("\n\n".join(summary_list))

            # word count
            word_element= article.find_element(By.CSS_SELECTOR,'dd.words')
            words.append(word_element.text)

            # chapter
            chapters_elements= article.find_element(By.CSS_SELECTOR, 'dd.chapters')
            chapters.append(chapters_elements.text)

            # view history
            latest_view_elements= article.find_element(By.CSS_SELECTOR, 'div.user.module.group h4.viewed.heading')
            latest_view.append(latest_view_elements.text)

        print(f"Found {len(authors)} authors on this page")

        print(f"Found {len(fandom_tags)} fandom tags on this page")

        print(f"Found {len(relationship_tags)} relationship tags on this page")

        print(f"Found {len(character_tags)} character tags on this page")

        print(f"Found {len(freeform_tags)} character tags on this page")

        print(f"Found summaries on this page")

        print(f"Found words on this page")

        print(f"Found chapter on this page")




        if current_page == max_pages:
            break

        try:
            # Add explicit wait for next button
            next_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, 'a[rel="next"]'))
            )
            print("Found next button, attempting to click")

            # Add a small delay before clicking
            time.sleep(2)

            next_button.click()
            print("Clicked next button")

            # Wait for next page to load
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'h4.heading'))
            )
            current_page += 1
            print(f"Successfully moved to page {current_page}")

        except Exception as e:
            print(f"Error occurred: {str(e)}")
            break

    df= pd.DataFrame({"Title": titles, 'Author': authors, "Fandom Tags": fandom_tags, "Relationship Tags":relationship_tags, "Character Tags":character_tags,
                      "Freeform Tags": freeform_tags, "Summary":summary, "Chapters": chapters, "Words": words, "Latest View": latest_view, "Link":links})

    return df

# Use the function
all_titles = scrape_titles_two_pages(driver)
print(f"\nTotal titles collected: {len(all_titles)}")


Scraping page 1
Found 20 titles on this page
Found 20 authors on this page
Found 20 fandom tags on this page
Found 20 relationship tags on this page
Found 20 character tags on this page
Found 20 character tags on this page
Found summaries on this page
Found words on this page
Found chapter on this page
Found next button, attempting to click
Clicked next button
Successfully moved to page 2
Scraping page 2
Found 20 titles on this page
Found 40 authors on this page
Found 40 fandom tags on this page
Found 40 relationship tags on this page
Found 40 character tags on this page
Found 40 character tags on this page
Found summaries on this page
Found words on this page
Found chapter on this page
Found next button, attempting to click
Clicked next button
Successfully moved to page 3
Scraping page 3
Found 20 titles on this page
Found 60 authors on this page
Found 60 fandom tags on this page
Found 60 relationship tags on this page
Found 60 character tags on this page
Found 60 character tags on thi

In [156]:
all_titles.to_csv("My_AO3_data.csv", index=False)

In [140]:
authors= driver.find_elements(By.CSS_SELECTOR, 'h4.heading a[rel="author"]')
authors= [a.text for a in authors]


articles = driver.find_elements(By.CSS_SELECTOR, 'li[role="article"]')
titles=[]
authors= []
fandom_tags = []
relationship_tags=[]
character_tags= []
freeform_tags= []
summary= []
words=[]
chapters=[]
latest_view= []

header_titles = driver.find_elements(By.CSS_SELECTOR, 'h4.heading a[href^="/works/"]')
for title in header_titles:
    titles.append(title.text)

for article in articles:
    title
    fandom_elements = article.find_elements(By.CSS_SELECTOR, 'h5.fandoms a')
    fandom_list= [fandom.text for fandom in fandom_elements]
    fandom_tags.append(", ".join(fandom_list))

    relationship_elements= article.find_elements(By.CSS_SELECTOR, 'li.relationships a')
    relationship_list= [relationship.text for relationship in relationship_elements]
    relationship_tags.append(", ".join(relationship_list))

    character_elements= article.find_elements(By.CSS_SELECTOR, 'li.characters a')
    character_list= [character.text for character in character_elements]
    character_tags.append(", ".join(character_list))

    author_elements= article.find_elements(By.CSS_SELECTOR, 'h4.heading a[rel="author"]')
    author_list= [author.text for author in author_elements]
    if not author_list:
      authors.append("Anonymous")
    else:
      authors.append(", ".join(author_list))

    freeform_elements= article.find_elements(By.CSS_SELECTOR, 'li.freeforms a')
    freeform_list= [freeform.text for freeform in freeform_elements]
    freeform_tags.append(", ".join(freeform_list))

    summary_elements= article.find_elements(By.CSS_SELECTOR, 'blockquote.userstuff p')
    summary_list= [summary.text for summary in summary_elements]
    summary.append("\n\n".join(summary_list))

    word_element= article.find_element(By.CSS_SELECTOR,'dd.words')
    words.append(word_element.text)

    chapters_elements= article.find_element(By.CSS_SELECTOR, 'dd.chapters')
    chapters.append(chapters_elements.text)

    latest_view_elements= article.find_element(By.CSS_SELECTOR, 'div.user.module.group h4.viewed.heading')
    latest_view.append(latest_view_elements.text)




print(latest_view)
df =pd.DataFrame({"Latest View": latest_view, "Titles": titles, 'Tags': fandom_tags, "Authors": authors, "Words": words,"Relationship Tag": relationship_tags, "Character Tag": character_tags, "Summary": summary,"Freeform Tag": freeform_tags})

['Last visited: 02 Jan 2025 (Update available.) Visited 2 times', 'Last visited: 02 Jan 2025 (Latest version.) Visited once', 'Last visited: 02 Jan 2025 (Latest version.) Visited once', 'Last visited: 02 Jan 2025 (Update available.) Visited 14 times', 'Last visited: 02 Jan 2025 (Minor edits made since then.) Visited 4 times', 'Last visited: 02 Jan 2025 (Update available.) Visited once', 'Last visited: 02 Jan 2025 (Latest version.) Visited 2 times', 'Last visited: 02 Jan 2025 (Latest version.) Visited 2 times', 'Last visited: 02 Jan 2025 (Update available.) Visited 2 times', 'Last visited: 02 Jan 2025 (Latest version.) Visited once', 'Last visited: 02 Jan 2025 (Latest version.) Visited 2 times', 'Last visited: 02 Jan 2025 (Latest version.) Visited 9 times', 'Last visited: 01 Jan 2025 (Minor edits made since then.) Visited 3 times', 'Last visited: 01 Jan 2025 (Minor edits made since then.) Visited once', 'Last visited: 01 Jan 2025 (Update available.) Visited 4 times', 'Last visited: 01 J

In [141]:
df

,Latest View,Titles,Tags,Authors,Words,Relationship Tag,Character Tag,Summary,Freeform Tag
0,Last visited: 02 Jan 2025 (Update available.) ...,Learn to love,Batman - All Media Types,Purple_Hyancith,"42,994",Bruce Wayne/Original Female Character(s),"Bruce Wayne, Batman, Nightwing, Dick Grayson, ...",Seven times Bruce Wayne's girlfriend tries to ...,"Emotional Hurt/Comfort, Found Family, Batfamil..."
1,Last visited: 02 Jan 2025 (Latest version.) Vi...,Morning cuddles,Kamisama Hajimemashita | Kamisama Kiss,pesto_p,400,Momozono Nanami/Tomoe,"Momozono Nanami, Tomoe (Kamisama Hajimemashita)",nanami should get out of bed and start prepari...,"Cuddling, Fluff, One Shot, tomoe tries to be a..."
2,Last visited: 02 Jan 2025 (Latest version.) Vi...,Stuck in a Comic Book,"Batman - All Media Types, Batman (Comics), Jus...",AshleyLookAtME,"58,826","Bruce Wayne/Reader, Bruce Wayne/Original Femal...","Original Female Character(s), Bruce Wayne, Jas...","A woman from our universe ends up in an alley,...","Self-Insert, Alternate Universe - Canon Diverg..."
3,Last visited: 02 Jan 2025 (Update available.) ...,Of OldStones,"A Song of Ice and Fire - George R. R. Martin, ...",MoonWitch96,"5,730","Joanna Lannister/Tywin Lannister, Original Fem...","OC - Character, Tywin Lannister, Joanna Lannis...",Arelei Lannister comes to the world in the how...,"Original Female Lannister, Worldbuilding, Slow..."
4,Last visited: 02 Jan 2025 (Minor edits made si...,Petra Calliope and the Issue with Having Fun,Harry Potter - J. K. Rowling,MabelLover,"78,007","Orion Black/Walburga Black, Cygnus Black/Druel...","Orion Black, Walburga Black, Cygnus Black, Dru...","""Don’t get caught, Reg had said, all the way b...","The Noble and Most Ancient House of Black, Sel..."
5,Last visited: 02 Jan 2025 (Update available.) ...,The Winter of Widows,"A Song of Ice and Fire - George R. R. Martin, ...",laughingnell,"583,885",,Original Characters,"When she awoke to a new life in Westeros, Ursu...","Self-Insert, Self-Indulgent, Winter is Coming ..."
6,Last visited: 02 Jan 2025 (Latest version.) Vi...,Scars,僕のヒーローアカデミア | Boku no Hero Academia | My Hero ...,sunny_dey,"66,100",Aizawa Shouta | Eraserhead/Yamada Hizashi | Pr...,"Aizawa Shouta | Eraserhead, Yamada Hizashi | P...",Shota and Hizashi’s relationship has become a ...,"Romance, Drama & Romance, Fluff and Angst, Dom..."
7,Last visited: 02 Jan 2025 (Latest version.) Vi...,McBadass Grey's Anatomy WTS,Grey's Anatomy,Fantastic_writer06,"5,974","Derek Shepherd/Original Female Character(s), M...","Derek Shepherd, Meredith Grey, George O'Malley...",Juliet Grey. Older fraternal twin sister to Me...,"Meredith Grey sister, strong woman, Bisexual A..."
8,Last visited: 02 Jan 2025 (Update available.) ...,New Start Grey's Anatomy,"Grey's Anatomy, Shameless (US)",Fantastic_writer06,"80,163","Mark Sloan/Original Female Character, Alex Kar...","Aelia Gallagher, Mark Sloan (Grey's Anatomy), ...",Aelia Gallagher daughter of Frank and Monica. ...,"Mark Sloan Lives (Grey's Anatomy), Grey Sloan ..."
9,Last visited: 02 Jan 2025 (Latest version.) Vi...,Brotherly Intervention,"DCU, Marvel Cinematic Universe",completelyhopeless,862,"Darcy Lewis/Jason Todd, Dick Grayson & Jason Todd","Darcy Lewis, Jason Todd, Dick Grayson","After Darcy and Jason fight, Dick gets pulled ...","Comment Fic, Community: comment_fic, Some Humor"
